# Implementation  of Decision Trees on Iris Dataset

In [195]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from math import log2

## Creating a class to implement the decision tree

In [196]:
class Node(object):
    def __init__(self,l):
        self.left=None
        self.right=None
        self.entropy=0.0
        self.total_count=0
        self.samples=[0,0,0]
        self.split_feature=""
        self.split_value=0.0
        self.cl=""
        self.leaf_node=False
        self.level=l
        self.gain_ratio=0.0
        
    def linsert(self,l):
        self.left=Node(l)
        return self.left
    
    def rinsert(self,l):
        self.right=Node(l)
        return self.right
    
    def set_leaf_node(self):
        self.leaf_node=True
    
    def set_vals(self,ent,count,c0,c1,c2):
        self.entropy=ent
        self.total_count=count
        self.samples=list([c0,c1,c2])
        if(c0>=c1 and c0>=c2):
            self.cl="Setosa"
        elif(c1>=c0 and c1>=c2):
            self.cl="Versicolor"
        else:
            self.cl="Virginica"
            
    def set_feat(self,f,mn,ratio):
        self.split_feature=f
        self.split_value=mn
        self.gain_ratio=ratio
        
    def print_node(self):

        print("Level = ",self.level)
        print("Entropy = ",self.entropy)
        print("Total Data Points = ",self.total_count)
        print("Count of each Class ",self.samples)
        print("Majority Class = ",self.cl)
        print("Gain Ratio = ",self.gain_ratio)
        if(self.leaf_node==False):
            print(self.split_feature,"<=",self.split_value)
        else:
            print("Leaf Node")
        print()

In [197]:
iris=datasets.load_iris()
X=iris.data
Y=iris.target

In [198]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [199]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [200]:
set(X[:,0])

{4.3,
 4.4,
 4.5,
 4.6,
 4.7,
 4.8,
 4.9,
 5.0,
 5.1,
 5.2,
 5.3,
 5.4,
 5.5,
 5.6,
 5.7,
 5.8,
 5.9,
 6.0,
 6.1,
 6.2,
 6.3,
 6.4,
 6.5,
 6.6,
 6.7,
 6.8,
 6.9,
 7.0,
 7.1,
 7.2,
 7.3,
 7.4,
 7.6,
 7.7,
 7.9}

### continuous values

In [201]:
feature=iris.feature_names
feature

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [202]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Y is the target variable with three classes where: 
* 0 - Setosa
* 1 - Versicolor
* 2 - Virginica

In [203]:
df_X=pd.DataFrame(X,columns=feature)
df_X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [204]:
df_Y=pd.DataFrame(Y,columns=['class'])
df_Y.head()

,class
0,0
1,0
2,0
3,0
4,0


In [205]:
len(df_Y)

150

In [206]:
def entropy(Y):
    #size of each class
    Y_0=len(Y.loc[(Y['class']==0)])
    Y_1=len(Y.loc[(Y['class']==1)])
    Y_2=len(Y.loc[(Y['class']==2)])

    count=len(Y)
    ent=0
    
    #calculating total entropy
    if Y_0 !=0:
        ent +=-((Y_0/count)*log2(Y_0/count))
    elif Y_1 !=0:
        ent+=-((Y_1/count)*log2(Y_1/count))
    elif Y_2!=0:
        ent+=-((Y_2/count)*log2(Y_2/count))

    return ent

In [207]:
def DecisionTrees(X,Y,feature,level,Tree):
    
    print("Level ",level)
    Y_0=len(Y.loc[(Y['class']==0)])
    print("Count of 0 ",Y_0)
    Y_1=len(Y.loc[(Y['class']==1)])
    print("Count of 1 ",Y_1)
    Y_2=len(Y.loc[(Y['class']==2)])
    print("Count of 2 ",Y_2)

    total_count=len(Y)
    ent=entropy(Y)
    print("Current entropy is ",ent)
    
    #setting the vales for class Node object
    Tree.set_vals(ent,total_count,Y_0,Y_1,Y_2)
    
    #base conditions
    
    ## pure node or no more feature to split on
    if len(set(Y['class']))==1 or len(feature)==0:
        Tree.set_leaf_node()
        print("Reached leaf node")
        print()
        return
    
    #for loop to calculate entropy of split on each feature
    max_gain=-10
    max_mn=0
    max_feat=""
    max_e1=max_e2=0
    
    for f in feature:
        
        mn=round(X[f].mean())
   
        X_1=X.loc[(X[f]<=mn)]
        ind1=X_1.index
        len_1=len(ind1)
        Y_1=Y.loc[ind1]
        e1=entropy(Y_1)
        
        X_2=X.loc[(X[f]>mn)]
        ind2=X_2.index
        len_2=len(ind2)
        Y_2=Y.loc[ind2,:]
        e2=entropy(Y_2)
        
        total_entropy=len_1/total_count*e1+len_2/total_count*e2

        info_gain=ent-total_entropy
        
        if(info_gain>max_gain):
            max_gain=info_gain
            max_feat=f
            max_X1=X_1
            max_X2=X_2
            max_Y1=Y_1
            max_Y2=Y_2
            max_mn=mn
            max_e1=e1
            max_e2=e2
    
    if len(max_Y1)==0 or len(max_Y2)==0:
        Tree.set_leaf_node()
        print("Reached Leaf Node")
        print()
        return
    
    print("ig=",max_gain)
    
    split_info=-len(max_X1)/len(X)*log2(len(max_X1)/len(X))-len(max_X2)/len(X)*log2(len(max_X2)/len(X))
    
    gain_ratio=max_gain/split_info
    
    print("gain ratio=",gain_ratio)
    print("Splitting on feature ",max_feat)
    print()
    

    Tree.set_feat(max_feat,mn,gain_ratio)

    feature.remove(max_feat)
    ft1=list(feature)
    ft2=list(feature)
    
    left=Tree.linsert(level+1)
    DecisionTrees(max_X1,max_Y1,ft1,level+1,left)
    
    right=Tree.rinsert(level+1)
    DecisionTrees(max_X2,max_Y2,ft2,level+1,right)
        
        

In [208]:
Tree=Node(0)

In [209]:
DecisionTrees(df_X,df_Y,feature,0,Tree)

Level  0
Count of 0  50
Count of 1  50
Count of 2  50
Current entropy is  0.5283208335737187
ig= 0.15966263561842597
gain ratio= 0.1609864215841086
Splitting on feature  sepal width (cm)

Level  1
Count of 0  8
Count of 1  42
Count of 2  33
Current entropy is  0.32530500543102886
ig= -0.157861754208021
gain ratio= -0.15867686011643523
Splitting on feature  sepal length (cm)

Level  2
Count of 0  8
Count of 1  28
Count of 2  9
Current entropy is  0.44299610601416445
ig= 0.04163696391876287
gain ratio= 0.045341558106526514
Splitting on feature  petal width (cm)

Level  3
Count of 0  8
Count of 1  7
Count of 2  0
Current entropy is  0.4836749843245433
ig= 0.4836749843245433
gain ratio= 0.4852317864697462
Splitting on feature  petal length (cm)

Level  4
Count of 0  8
Count of 1  0
Count of 2  0
Current entropy is  0.0
Reached leaf node

Level  4
Count of 0  0
Count of 1  7
Count of 2  0
Current entropy is  0.0
Reached leaf node

Level  3
Count of 0  0
Count of 1  21
Count of 2  9
Current 

In [147]:
def Print_Tree(Tree):
    if(Tree==None):
        return
    
    Tree.print_node()
    Print_Tree(Tree.left)
    Print_Tree(Tree.right)

In [148]:
Print_Tree(Tree)

Level =  0
Entropy =  0.5283208335737187
Total Data Points =  150
Count of each Class  [50, 50, 50]
Majority Class =  Setosa
sepal width (cm) <= 1

Level =  1
Entropy =  0.32530500543102886
Total Data Points =  83
Count of each Class  [8, 42, 33]
Majority Class =  Versicolor
sepal length (cm) <= 1

Level =  2
Entropy =  0.44299610601416445
Total Data Points =  45
Count of each Class  [8, 28, 9]
Majority Class =  Versicolor
petal width (cm) <= 1

Level =  3
Entropy =  0.4836749843245433
Total Data Points =  15
Count of each Class  [8, 7, 0]
Majority Class =  Setosa
petal length (cm) <= 2

Level =  4
Entropy =  0.0
Total Data Points =  8
Count of each Class  [8, 0, 0]
Majority Class =  Setosa
Leaf Node

Level =  4
Entropy =  0.0
Total Data Points =  7
Count of each Class  [0, 7, 0]
Majority Class =  Versicolor
Leaf Node

Level =  3
Entropy =  0.3602012209808308
Total Data Points =  30
Count of each Class  [0, 21, 9]
Majority Class =  Versicolor
petal length (cm) <= 4

Level =  4
Entropy 